In [1]:
import octomap

In [2]:
print(dir(octomap))

['NullPointerException', 'OcTree', 'OcTreeKey', 'OcTreeNode', '__builtins__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__test__', '_octree_read', 'iterator_base', 'leaf_bbx_iterator', 'leaf_iterator', 'np', 'tree_iterator']


In [3]:
import pyrealsense2 as rs
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
from mpl_toolkits.mplot3d import Axes3D
import time

# Collecting frames for next dev

In [4]:
count_of_frames = 20
frames_between_frames_D435 = 15

In [5]:
count_of_frames * frames_between_frames_D435 / 30

10.0

# Iitial configs

In [6]:
# data filenames
d435_filename = '../data/D435.bag'

In [7]:
# config fo D435
cfg_d435 = rs.config()
cfg_d435.enable_device_from_file(d435_filename)
cfg_d435.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
pipe_d435 = rs.pipeline()

## Collecting frames

In [8]:
d435_frames = []
pipe_d435.start(cfg_d435)
while len(d435_frames) != count_of_frames:
    for i in range(frames_between_frames_D435):
        depth_frame = pipe_d435.wait_for_frames().get_depth_frame()
    d435_frames.append(depth_frame)
pipe_d435.stop()

In [9]:
def count_diff_frames(frames):
    prev_frame = None
    for i, frame in enumerate(frames):
        if prev_frame is not None:
            print("Difference bw {0} and {1} frames is {2:.4f} ms".format(i, i-1, frame.get_timestamp()-prev_frame.get_timestamp()))
        prev_frame = frame

In [10]:
count_diff_frames(d435_frames)

Difference bw 1 and 0 frames is 500.7671 ms
Difference bw 2 and 1 frames is 499.8582 ms
Difference bw 3 and 2 frames is 500.9062 ms
Difference bw 4 and 3 frames is 499.7070 ms
Difference bw 5 and 4 frames is 500.0930 ms
Difference bw 6 and 5 frames is 500.2417 ms
Difference bw 7 and 6 frames is 500.3752 ms
Difference bw 8 and 7 frames is 500.2996 ms
Difference bw 9 and 8 frames is 500.2456 ms
Difference bw 10 and 9 frames is 500.3613 ms
Difference bw 11 and 10 frames is 500.2937 ms
Difference bw 12 and 11 frames is 500.7803 ms
Difference bw 13 and 12 frames is 499.8145 ms
Difference bw 14 and 13 frames is 500.7185 ms
Difference bw 15 and 14 frames is 499.9082 ms
Difference bw 16 and 15 frames is 500.3823 ms
Difference bw 17 and 16 frames is 500.2190 ms
Difference bw 18 and 17 frames is 500.4111 ms
Difference bw 19 and 18 frames is 500.2153 ms


In [11]:
depth_frame = d435_frames[0]
depth_frame

In [12]:
def get_coordinates(depth_frame, make_sampling=True, koef = 2**2, count_sampling = 1):
    """
     TODO
    :param make_sampling:
    :return:
    """
    pc = rs.pointcloud()
    if make_sampling:
        decimate = rs.decimation_filter()
        decimate.set_option(rs.option.filter_magnitude, koef)
        for i in range(count_sampling):
            depth_frame = decimate.process(depth_frame)
        points = pc.calculate(depth_frame).as_points()
    else:
        points = pc.calculate(depth_frame).as_points()

    coordinates = np.ndarray(buffer=points.get_vertices(), dtype=np.float32, shape=(points.size(), 3))
    coordinates = coordinates[coordinates[:, 2] != 0]

#     coordinates = apply_transformation(tm_T265toD435, coordinates)
    return coordinates


In [13]:
pc = get_coordinates(depth_frame, koef = 2 ** 1, count_sampling = 1)

In [14]:
pc.shape

(91959, 3)

# Octomap

In [15]:
import math3d as m3d
import glooey
import imgviz
import numpy as np
import pyglet
import trimesh
import trimesh.transformations as tf
import trimesh.viewer

In [16]:
m3d.Transform(np.zeros(6)).array

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [17]:
def labeled_scene_widget(scene, label):
    vbox = glooey.VBox()
    vbox.add(glooey.Label(text=label, color=(255, 255, 0)), size=0)
    vbox.add(trimesh.viewer.SceneWidget(scene))
    return vbox


In [18]:
def visualize(occupied, resolution, aabb):
    window = pyglet.window.Window( # pyglet http://pyglet.org/
        width=int(640), height=int(480)
    )

    @window.event
    def on_key_press(symbol, modifiers):
        if modifiers == 0:
            if symbol == pyglet.window.key.Q:
                window.on_close()

    gui = glooey.Gui(window)
    hbox = glooey.HBox()
    hbox.set_padding(5)

    aabb_min, aabb_max = aabb
    bbox = trimesh.path.creation.box_outline(
        aabb_max - aabb_min,
        tf.translation_matrix((aabb_min + aabb_max) / 2),
    )

    geom = trimesh.voxel.ops.multibox(
        occupied, pitch=resolution, colors=[1., 0, 0, 0.5]
    )
    scene = trimesh.Scene(geometry=[bbox, geom])
    hbox.add(labeled_scene_widget(scene, label='occupied'))

    geom = trimesh.voxel.ops.multibox(
        empty, pitch=resolution, colors=[0.5, 0.5, 0.5, 0.5]
    )
#     scene = trimesh.Scene(geometry=[bbox, geom])
#     hbox.add(labeled_scene_widget(scene, label='empty'))

    gui.add(hbox)
    pyglet.app.run()

In [19]:
resolution = 0.1
octree = octomap.OcTree(resolution)

octree.insertPointCloud(
    pointcloud=np.double(pc),
    origin=np.array([0, 0, 0], dtype=float),
    maxrange=2,
)


occupied, empty = octree.extractPointCloud()

aabb_min = octree.getMetricMin()
aabb_max = octree.getMetricMax()

visualize(
    occupied=occupied, # no :(
    resolution=resolution,
    aabb=(aabb_min, aabb_max),
)
